In [1]:
!pip install openai

In [8]:
!pip install AudioSegment


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73501 sha256=1f437e1dac2d3b04909a811f71069e73215db21b157c1637e8b419d147f788db
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad


In [9]:
import os


In [10]:
def compress_video_for_api(input_path):
    # Extract and heavily compress audio
    audio = AudioSegment.from_file(input_path)
    
    # Aggressive compression to fit 25MB limit
    audio = audio.set_frame_rate(8000)  # Lower sample rate
    audio = audio.set_channels(1)       # Mono
    
    compressed_path = "/tmp/ultra_compressed.mp3"
    audio.export(compressed_path, 
                format="mp3", 
                bitrate="32k",  # Very low bitrate
                parameters=["-ac", "1"])  # Force mono
    
    size_mb = os.path.getsize(compressed_path) / (1024 * 1024)
    print(f"Compressed size: {size_mb:.1f}MB")
    
    if size_mb <= 25:
        return compressed_path
    else:
        print("Still too large even after compression")
        return None
    
video = compress_video_for_api("/kaggle/input/vaibhavee/2025-08-07 18-02-57.mkv")
print(video)
    

Compressed size: 5.5MB
/tmp/ultra_compressed.mp3


In [ ]:
import openai
from pydub import AudioSegment
import os

# Initialize client
from kaggle_secrets import UserSecretsClient
user_secrets = openai.util.UserSecrets()
api_key = user_secrets.get_secret("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

# file_path = "/kaggle/input/vaibhavee/2025-08-07 18-02-57.mkv"

def transcribe_audio_safe(file_path):
    try:
        
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return None
        
        # Check file size and compress if needed
        file_size = os.path.getsize(file_path) / (1024 * 1024)
        print(file_size)
        print(file_path)
                
        # Transcribe with proper file handling
        with open(file_path, "rb") as audio_file:
            transcription = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                response_format="text"
            )
        
        #save transcription in kaggle working directory
        with open("/kaggle/working/transcription_result.txt", "w", encoding="utf-8") as f:
            f.write(transcription)
        # return transcription
        
    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

# Usage
file_path = "/kaggle/input/vaibhavee/2025-08-07 18-02-57.mkv"
result = transcribe_audio_safe("/tmp/ultra_compressed.mp3")

print(result)

5.533032417297363
/tmp/ultra_compressed.mp3
Good evening, Babavi. I am your host today. My name is Ayushi Dixit, and today we'll let everyone know about your journey to know how it is impressive and inspiring to all of us. So can we start with your brief introduction, your background, and what you currently do? Yeah. Hello, Ayushi. So I am Babavi. I am from Sitamarhi, Bihar, and I am pursuing this BS in Electronic Systems. So I started this degree in 7th, and currently I am in the first term of degree level. First term of degree level, right? Yeah. Yeah. So are you doing this degree right after your 12th? Yes. So how did you came to know about this degree and what were the initial steps when you came to know and decided to join? Yeah. So I came to know through some YouTube channels about this degree, and then I, at that time I was stressed because of the, like how to move forward after that, you know. So then I decided to take this degree. Before joining this degree, let's rewind a lit

In [ ]:
# text to blog generation
import openai
from IPython.display import display, Markdown
import os
# Initialize client

def generate_blog_from_text(transcription, max_retries=3):
    
    prompt = f"""
    You are a professional blog writer specializing in educational content. Transform the following transcription into an engaging, well-structured 700 words blog post about BS DEGREE ELECTRONICS PROGRAMS. 
    Ensure the blog is engaging, creative, informative about the ELECTRONICS PROGRAM, and easy to read. Write in a single, coherent narrative like a story. highlight fee waiver information if any. 
    dont use the word "podcast" in the blog. 
    Include practical advice and insights for students
    dont mention any private instituton names.
    Include relevant quotes from the discussion
    mention the incidents of students meeting with professsors and how it helped them.
    
    Transcription:
    {transcription}
    
    Create an inspiring 700-word blog post:
    """
    
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt + 1} of {max_retries}")
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an expert blog writer who creates engaging educational content from transcriptions. Focus on storytelling and practical value for students."},
                    {"role": "user", "content": prompt}                
                ],  
                max_tokens=2000,
                temperature=0.7,
                top_p=1,
                n=1,
                # stop=None,
                frequency_penalty=0.1,
                presence_penalty=0.1
            )
            blog_content = response.choices[0].message.content
            print("blog generated")
            return blog_content
            
           
        except openai.RateLimitError:
            print("Rate limit exceeded. Retrying...")
            continue
    return None
# Read transcription from file
with open("/kaggle/working/transcription_result.txt", "r", encoding="utf-8") as f:
    transcription = f.read()
    
print(f" Transcription length: {len(transcription)} characters")
   
# Generate blog post    
blog_post = generate_blog_from_text(transcription)

# Display blog post
display(Markdown(blog_post))

# save blog in kaggle working directory
with open("/kaggle/working/generated_blog_post.md", "w", encoding="utf-8") as f:
    f.write(blog_post)  
    
print("Blog post generated")
    






📝 Transcription length: 19699 characters
Attempt 1 of 3
blog generated


**Navigating the World of Electronics: Babavi's Journey through a BS Degree Program**

In the quaint town of Sitamarhi, Bihar, a young student named Babavi sat in front of her computer, her eyes glued to a YouTube channel that could potentially define her future. Like many students at the crossroads of high school completion and college, Babavi was filled with uncertainty about the path she should take. But one thing was clear—her interest in electronics was not just a fleeting fancy but a genuine passion she wanted to pursue further.

Fast forward to the present, Babavi is now a proud student pursuing a Bachelor of Science in Electronic Systems, a program filled with both challenges and opportunities. Her journey through this degree is not just about academic growth; it is a story of transformation and resilience.

Babavi's initial interest in electronics was sparked during her schooling in Sitamarhi. Although she had qualified for JEE Mains, her percentile wasn’t enough to secure a seat in the top-tier engineering colleges. When faced with limited options, Babavi chose an online degree in Electronic Systems offered by a prestigious institution known for its innovative programs in data science and electronics.

"The program is structured really well," Babavi shares, her voice brimming with enthusiasm. "We are studying from professors who are experts in their fields. The live sessions, TA sessions, and recorded videos make learning accessible and comprehensive."

One of the most enlightening aspects of the program has been the opportunity to interact with professors and teaching assistants. Babavi recalls how instructor sessions played a crucial role in overcoming complex subjects like control engineering. "These sessions are invaluable for course preparation and doubt-solving," she adds. As a teaching assistant herself, Babavi finds joy in helping fellow students navigate through their academic challenges.

Despite the online setup, Babavi has found ways to engage and grow beyond the confines of her screen. Participating in online events and being part of the organizing team for Paradox 2024 helped her shed her introverted shell. "Being part of the core team for event organizing taught me leadership skills and boosted my confidence," she reflects.

Practical hands-on projects form the core of Babavi's learning experience. She fondly remembers working under a professor on a project to build a graphic equalizer. "Working directly with a professor provided a deeper insight into practical applications," she notes. Such projects are not merely academic exercises; they are gateways to understanding real-world applications and industry-aligned skills.

Despite the challenges, Babavi's journey is also marked by financial support that made her education possible. "I am not the only child in my family, so managing finances was tough," she explains. Thanks to a fee waiver she discovered through the same YouTube channel that inspired her path, Babavi could afford her education without burdening her family. Additionally, she received a scholarship after completing her foundation level, which eased her financial constraints further.

The program's alignment with industry standards is another highlight. "From this term, they are sending us for internships that count towards our degree credits," Babavi mentions excitedly. Real-world exposure through internships and projects is invaluable for students aspiring to enter the workforce.

Babavi's interactions with professors have left a lasting impact on her. She speaks highly of Janki Ramanathan, her diploma-level project guide, whose mentorship enriched her learning experience. "The professors do not differentiate us from their regular students," she remarks, emphasizing the inclusive nature of the program.

As Babavi looks towards the future, she remains open to various career paths. Whether it's hopping into a job or pursuing further studies, one thing is certain—her degree has equipped her with skills and confidence to tackle whatever comes next.

For students like Babavi who are contemplating an online degree in electronics, she offers this advice: "Don't be deterred by the online format. The content is robust, and if you study with focus, you'll do exceptionally well." Her story is a testament to how determination, coupled with the right opportunities, can lead to success.

Babavi’s journey is more than just an academic pursuit; it’s an inspiring tale of self-discovery and perseverance—a reminder that sometimes the unconventional path can lead to extraordinary destinations.

Blog post generated
